In [1]:
# Load the packages and the Model!!
from sentence_transformers import SentenceTransformer, util
import torch
from sklearn.cluster import KMeans

# Load a pre-trained SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast & accurate

/home/yonas/Venvs/torch_venv/lib/python3.9/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
###########  Semantic Textual Similarity ###################
"""
Compare the meaning of two sentences.
"""

sentence1 = "A man is playing a guitar"
sentence2 = "A person is performing music"

# Encode sentences
emb1 = model.encode(sentence1, convert_to_tensor=True)
emb2 = model.encode(sentence2, convert_to_tensor=True)

# Compute cosine similarity
similarity = util.pytorch_cos_sim(emb1, emb2)
print(f"Similarity: {similarity.item():.4f}")

Similarity: 0.6417


In [5]:
######### Semantic Search (Information Retrieval)  ##################
"""
Find the most relevant documents from a list.
"""
corpus = [
    "The cat sits outside",
    "A man is playing guitar",
    "I love pizza",
    "The new movie is amazing",
    "I have a cat who loves music"
]
query = "A person playing an instrument"

# Encode
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
query_embedding = model.encode(query, convert_to_tensor=True)

# Search
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=3)[0]
for hit in hits:
    print(f"{corpus[hit['corpus_id']]} (Score: {hit['score']:.4f})")

A man is playing guitar (Score: 0.6251)
I have a cat who loves music (Score: 0.3785)
The cat sits outside (Score: 0.0595)
